# Extract Author Information

We want to obtain year of birth and year of death for authors in the DBNL dataset.

There is a database of author metadata on DBNL that I've downloaded (http://dbnl.nl/data, for login details, ask Peter).

This notebook merges the book metadata and the author metadata.

The author metadata has four columns:


| ID | Name | Volledige naam | Personalia |
| --- | --- | --- | --- |
| aafj001 | Aafjes, Bertus | [Lambertus Jacobus Johannes] | (Amsterdam 12 mei 1914 - Swolgen 22 april 1993) |
| aals010 | Aalst, George van |  | (1897 - 1925) |
| a___002 | A, Arien |  | (?(17de eeuw)) |
| a___001 | A, Herman van der |  | (Nieuw Weerdinge 1950) |

The column `Personalia` contains `DoB` and `DoD` information, where possible including geographic location and exact date of birth or death. But in many cases, either or both are unknown or only partially known. In the case of `A, Herman van der`, it is not clear if `Nieuw Weerdingen 1950` is the location and year of birth or death. To check, we should compare this year with the years of publication of titles by _Herman van der A_. If those publications are before 1950, then this is the year of death. If they are after 1950, it is the date of birth.

So we start with the book title metadata and compute the year of starting the first book and finishing the last book as the period that they were active. 

In [1]:
import re
import pandas as pd

publication_file = '../metadata/DBNL-single_author_prose_titles.tsv'
pub_df = pd.read_csv(publication_file, sep='\t')

We need to keep in mind that some titles have no known author (`Anoniem`) or not a single/stable (set of) of authors, such as magazines, where the contributors may change between issues.

In [2]:
# if `Voornaam` is `Anoniem`, the author is anonymous
pub_df['is_anonymous'] = pub_df.voornaam.apply(lambda x: True if x == 'Anoniem' else False)
print(f"number of titles with anonymous authors: {len(pub_df[pub_df.is_anonymous == True])}")

# if the `ti_id` starts with an underscore, there is no seperate author ID
pub_df['no_author_id'] = pub_df.ti_id.str.startswith('_')
print(f"number of titles with no author ID: {len(pub_df[pub_df.no_author_id == True])}")


number of titles with anonymous authors: 0
number of titles with no author ID: 0


In [3]:
pub_df.shape

(2069, 24)

## Extracting original print year for reprints

Some of the titles are reprints. To know where a work sits on the timeline of the authors active period, we need to know the year of the original publication. A separate spreadsheet with reprint information can be used to find the original publication year for many titles.

In [4]:
reprint_file = '../metadata/DBNL 20250122 Proza 19e en 20e eeuw later herdrukt.xlsx'

reprint_df = pd.read_excel(reprint_file)
reprint_df.head(2)

,origineel.ti_id,origineel.titel,origineel._jaar,origineel.druk,origineel.categorie,origineel.genre,latere_uitgave.ti_id,latere_uitgave.titel,latere_uitgave._jaar,latere_uitgave.druk,latere_uitgave.categorie,geplaatst onder,latere_uitgave.genre
0,sieg002korr01,"Twee redevoeringen, gevolgd van aanmerkingen, ...",1800,1ste druk,werk,proza,sieg002rede01,Redevoering over het openbaar onderwijs in de ...,1997,1ste druk,uitgave,sieg002rede01_01,proza
1,sieg002korr01,"Twee redevoeringen, gevolgd van aanmerkingen, ...",1800,1ste druk,werk,proza,sieg002rede01,Redevoering over het openbaar onderwijs in de ...,1997,1ste druk,uitgave,sieg002rede01_01,sec - taalkunde


In [5]:
reprint_df['origineel._jaar'].describe()

count    1650.000000
mean     1913.807879
std        39.856094
min      1800.000000
25%      1887.000000
50%      1915.500000
75%      1940.000000
max      1999.000000
Name: origineel._jaar, dtype: float64

The original publication year column contains integers, with the minimum being 1800 and the maximum 1999, so the column can be used directly. 

In [6]:
reprint_id_field = 'latere_uitgave.ti_id'
reprint_map = reprint_df[[reprint_id_field, 'origineel._jaar']].drop_duplicates()
s = reprint_df[reprint_id_field].value_counts()
s[s > 1]


latere_uitgave.ti_id
_sti001stij03    36
mara002vers01    21
_sti001stij02    12
liev040paul02    12
verm036verz03    12
                 ..
bril009waar01     2
spyr001klei08     2
burn005klei05     2
vrie229janh01     2
bosb002dona01     2
Name: count, Length: 358, dtype: int64

There are reprints that have multiple rows in the spreadsheet. What is going on here?

In [7]:
reprint_df[reprint_df[reprint_id_field].isin(s[s > 3].index)].head(5)

,origineel.ti_id,origineel.titel,origineel._jaar,origineel.druk,origineel.categorie,origineel.genre,latere_uitgave.ti_id,latere_uitgave.titel,latere_uitgave._jaar,latere_uitgave.druk,latere_uitgave.categorie,geplaatst onder,latere_uitgave.genre
21,dros003augu01,De Augustusdagen,1833,handschrift,werk,proza,dros003sche01,Schetsen en verhalen,1835,1ste druk,werk,dros003sche01_01,proza
22,dros003alta01,Het Altaarstuk,1833,1ste druk,werk,proza,dros003sche01,Schetsen en verhalen,1835,1ste druk,werk,dros003sche01_01,proza
23,dros003meer01,Meerhuyzen,1834,1ste druk,werk,proza,dros003sche01,Schetsen en verhalen,1835,1ste druk,werk,dros003sche01_01,proza
28,dros003pest01,De pestilentie te Katwijk (1625),1835,1ste druk,werk,proza,dros003sche01,Schetsen en verhalen,1835,1ste druk,werk,dros003sche01_01,proza
45,cons001sisk01,Siska van Roosemael,1841,1ste druk,werk,proza,cons001voll32,Volledige werken 32. Eenige bladzijden uit het...,1912,1ste druk,uitgave,cons001voll32_01,proza


Some titles in our dataset map back to multiple earlier prints. This is the case for collections of stories of an author, where the publication year of the collection maps back to the years that the individual stories were published. What we can do is map these titles to entire period, and in a later step, split these into the start year and end year.

In [8]:
reprint_map = (reprint_df
               .groupby(reprint_id_field)['origineel._jaar']
               .describe()[['min', 'max']]
               .apply(lambda row: f"{row['min']:.0f}-{row['max']:.0f}", axis=1)
               .rename('orig_jaar')
               .reset_index()
               .rename(columns={'latere_uitgave.ti_id': 'reprint_id'}))

reprint_map

,reprint_id,orig_jaar
0,_alp001alph01,1830-1830
1,_bea001beat38,1982-1982
2,_dri019drie02,1925-1925
3,_for003193201,1933-1933
4,_int001inte01,1927-1928
...,...,...
994,zett001voll01,1847-1854
995,zikk001wees02,1966-1966
996,zuyl002gesc01,1845-1845
997,zuyl002mijn01,1985-1985


In [9]:
pub_ext_df = pd.merge(pub_df, reprint_map, left_on='ti_id', right_on='reprint_id', how='left')

# pub_df and pub_ext_df should have the same number of rows
pub_df.shape, pub_ext_df.shape

((2069, 24), (2069, 26))

In [10]:
print("numer of titles with mapped original year:", len(pub_ext_df[pub_ext_df.orig_jaar.notna()]))
print("numer of titles with unmapped original year:", len(pub_ext_df[pub_ext_df.orig_jaar.isna()]))

numer of titles with mapped original year: 596
numer of titles with unmapped original year: 1473


In [11]:
pub_ext_df['best_jaar'] = (pub_ext_df.apply(lambda row: row['jaar'] if pd.isna(row['orig_jaar']) else row['orig_jaar'], axis=1))
pub_df = pub_ext_df

## Title Identifier Check

In [12]:
tid_freq = pub_df.ti_id.value_counts()
tid_freq[tid_freq > 1]

Series([], Name: count, dtype: int64)

## Extracting author ID and years starting and ending a publication

The title ID in most cases consist of an author ID and an identifier of one of their publications. I.e. `woen003lant04` has the author ID `woen003` for _Peter van Woensel_ and `lant04` is the specifier for _De Lantaarn voor 1800_. So the specifier has the first four letters of the first content word of the title and a running number to separate multiple publications starting with the same four letters. 

The `Jaar` column contains an indication of the year(s) of writing or publication (I'm not sure how to exactly interpret this). What I think we're interested in is the range from first to last years to establish the period of author activity. 

In [13]:
def extract_author_id(title_id):
    if title_id.startswith('_'):
        return None
    if m := re.match(r"^(\w+\d+)(\w+\d+)", title_id):
        author_id = m.group(1)
    else:
        print('unexpected title_id format:', title)
    return author_id


def extract_year(jaar):
    jaar = jaar.strip()
    year_start, year_end, precision = None, None, None
    if m := re.match(r"^\d{4}$", jaar):
        year_start = int(jaar)
        year_end = year_start
        precision = 'exact'
    elif m := re.match(r"^(\d{4})-(\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = int(m.group(2))
        precision = 'exact'
    elif m := re.match(r"^ca\. (\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = year_start
        precision = 'circa'
    elif m := re.match(r"^ca\. (\d{4})\?$", jaar):
        year_start = int(m.group(1))
        year_end = year_start
        precision = 'circa'
    elif m := re.match(r"^na (\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = year_start
        precision = 'after'
    elif m := re.match(r"^ca\. (\d{4})-(\d{4})$", jaar):
        year_start = int(m.group(1))
        year_end = int(m.group(2))
        precision = 'circa'
    else:
        print(f"#{jaar}#")
    return year_start, year_end, precision

pub_df['author_id'] = pub_df.ti_id.apply(extract_author_id)
pub_df.author_id
pub_df['year_start'], pub_df['year_end'], pub_df['year_precision'] = zip(*pub_df.best_jaar.apply(extract_year))

pub_df

,ti_id,titel,jaar,druk,categorie,voornaam,voorvoegsel,achternaam,uitgever,plaats_van_uitgave,...,year_precision,decade_start,decade_end,is_anonymous,no_author_id,is_magazine,is_prose,reprint_id,orig_jaar,best_jaar
0,kist001leve01,"Het leven, gevoelens en zonderlinge reize van ...",1800,1ste druk,werk,Willem,NaN,Kist,NaN,Haarlem,...,exact,1800,1800,False,False,False,True,NaN,NaN,1800
1,stre001char01,"Charakters en lotgevallen van Adelson, Héloïse...",1804,1ste druk,werk,Naatje,van,Streek-Brinkman,Gerbrand Roos,Amsterdam,...,exact,1800,1800,False,False,False,True,NaN,NaN,1804
2,daal002will01,Willem Hups. Eene anecdote uit de XVII eeuw; o...,1805,1ste druk,werk,Bruno,NaN,Daalberg,J. Immerzeel Jr.,Den Hage,...,exact,1800,1800,False,False,False,True,NaN,NaN,1805
3,bodd002leve01,Levensgeschiedenis van den vermaarden dichter,1805,1ste druk,uitgave,Pieter,NaN,Boddaert jr.,NaN,NaN,...,exact,1800,1800,False,False,False,True,NaN,NaN,1805
4,loos005hist01,Historie van Mejuffrouw Susanna Bronkhorst. De...,1806,1ste druk,werk,Adriaan,NaN,Loosjes,NaN,NaN,...,exact,1800,1800,False,False,False,True,NaN,NaN,1806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,merw017coff02,Coffy de negerslaaf,ca. 1970,2de druk,uitgave,H.,te,Merwe,W.D. Meinema,Delft,...,exact,1970,1970,False,False,False,True,merw017coff02,1966-1966,1966-1966
2065,aard002stij01,Stijfkopje omnibus (onder pseudoniem Ankie Aal...,ca. 1970,herdruk,herdruk,H.P.,van den,Aardweg,Geka,Alphen aan de Rijn,...,circa,1970,1970,False,False,False,True,NaN,NaN,ca. 1970
2066,gras003daar01,"Daar ga je, Claudia!",ca. 1970-1980,?,werk,Cok,NaN,Grashoff,Het Goede Boek / Standaard,Huizen / Antwerpen,...,circa,1970,1980,False,False,False,True,NaN,NaN,ca. 1970-1980
2067,nijn001tolh02,"Tolhuis ""Het zwarte paard""",na 1941,herdruk,herdruk,H.J.,van,Nijnatten-Doffegnies,Westfriesland,Hoorn,...,exact,1940,1940,False,False,False,True,nijn001tolh02,1941-1941,1941-1941


Check the number of titles per author:

In [14]:
author_freq = pub_df.author_id.value_counts()
author_freq

author_id
snie001        45
bosb002        34
kiev001        32
coup002        31
reyn008        31
               ..
wage010         1
kort010         1
berg063         1
revi0028100     1
aard002         1
Name: count, Length: 741, dtype: int64

In [15]:
for i in range(1, 11):
    print(f"number of authors with at least {i} titles: {len(author_freq[author_freq >= i])}")

number of authors with at least 1 titles: 741
number of authors with at least 2 titles: 316
number of authors with at least 3 titles: 198
number of authors with at least 4 titles: 138
number of authors with at least 5 titles: 103
number of authors with at least 6 titles: 85
number of authors with at least 7 titles: 68
number of authors with at least 8 titles: 56
number of authors with at least 9 titles: 44
number of authors with at least 10 titles: 36


So there are 82 authors with at least 10 titles (but this includes non-fiction publications like magazines and articles).

In [16]:
author_active = pd.concat([
    pub_df.groupby('author_id').year_start.min(), 
    pub_df.groupby('author_id').year_end.max()], axis=1)

author_active = (author_active
                 .rename(columns={'year_end': 'active_end', 'year_start': 'active_start'}))

author_active

,active_start,active_end
author_id,,
aalb003,1955,1955
aals010,1924,1924
aard002,1970,1970
abee001,1866,1874
abko001,1907,1931
...,...,...
zikk001,1968,1968
zome002,1966,1967
zutf001,1948,1948


Check that all authors have an explicit active period (there should be no `NaN`s):

In [17]:
author_active[author_active.active_start.isna()]

,active_start,active_end
author_id,,


## Parsing the author metadata

The author data is downloaded as a large (35Mb) HTML file with multiple `table` elements.

In [18]:
import numpy as np

from bs4 import BeautifulSoup as Bsoup

author_file = '../data/dbnl-auteurs.html'

with open(author_file, 'rt') as fh:
    page_soup = Bsoup(fh)

In [19]:
def rewrite_errors(date_string):
    for error in rewrite_map:
        if re.search(r'\b' + error + r'\b', date_string):
            date_string = re.sub(r'\b' + error + r'\b', rewrite_map[error], date_string)
    return date_string
    

def parse_date_string(date_string, prefix: str):
    date_string = rewrite_errors(date_string)
    date_info = {f'{prefix}_string': date_string}
    if m := re.match(r"^(.*) (\d{1,2}) (\w+) (\d{4})", date_string):
        date_info[f'{prefix}_location'] = m.group(1)
        date_info[f'{prefix}_day'] = int(m.group(2))
        date_info[f'{prefix}_month'] = month_map[m.group(3)]
        date_info[f'{prefix}_year'] = int(m.group(4))
    elif m := re.match(r"^(.*?) (circa|ca\.) (\d{4})", date_string):
        date_info[f'{prefix}_location'] = m.group(1)
        date_info[f'{prefix}_year'] = int(m.group(3))
    elif m := re.match(r"^(.*?) (circa|ca\.) " + month_pattern + " (\d{4})", date_string):
        date_info[f'{prefix}_location'] = m.group(1)
        date_info[f'{prefix}_month'] = month_map[m.group(3)]
        date_info[f'{prefix}_year'] = int(m.group(4))
    elif m := re.match(r"^(.*?) " + month_pattern + " (\d{4})", date_string):
        date_info[f'{prefix}_location'] = m.group(1)
        date_info[f'{prefix}_month'] = month_map[m.group(2)]
        date_info[f'{prefix}_year'] = int(m.group(3))
    elif m := re.match(r"^(.*?) (\d{4})", date_string):
        date_info[f'{prefix}_location'] = m.group(1)
        date_info[f'{prefix}_year'] = int(m.group(2))
    elif m := re.match(r"^(\d{4})$", date_string):
        date_info[f'{prefix}_year'] = int(m.group(1))
    elif m := re.match(r"^\?\((\d{2})(ste|de) eeuw\)$", date_string):
        date_info[f'{prefix}_century'] = int(m.group(1))
    year = f'{prefix}_year'
    century = f'{prefix}_century'
    if year in date_info and century not in date_info:
        date_info[century] = np.ceil(date_info[year] / 100)
    # ?(20ste eeuw)
    # Rotterdam ca. 1686 
    # Correggio circa augustus 1489
    return date_info
        
def parse_personalia(row_string):
    pers_string = row_json['Personalia']
    if pers_string.startswith('(') and pers_string.endswith(')'):
        pers_string = pers_string[1:-1]
    parts = [part.strip() for part in pers_string.split(' - ')]
    pers_info = {}
    if len(parts) == 2:
        dob_string, dod_string = parts
        dob_info = parse_date_string(dob_string, 'dob')
        dod_info = parse_date_string(dod_string, 'dod')
        pers_info.update(dob_info)
        pers_info.update(dod_info)
    elif len(parts) == 1:
        dob_string = parts[0]
        dob_info = parse_date_string(dob_string, 'dob')
        if row_string['active_start'] is not None and 'dob_year' in dob_info and row_string['active_start'] < dob_info['dob_year']:
            print('switching dob to dod', pers_string)
            dob_info = parse_date_string(dob_string, 'dod')
        pers_info.update(dob_info)
    # print(pers_string)
    # print(pers_info)
    # print()
    return pers_info
        

def get_active_years(author_id, author_active):
    active_start, active_end = None, None
    if author_id in author_active.index:
        active_start = author_active.loc[author_id].active_start
        active_end = author_active.loc[author_id].active_end
    #print(author_id, active_start, active_end)
    return active_start, active_end



# Map month names to numbers
month_map = {
    "januari": 1,
    "februari": 2,
    "maart": 3,
    "april": 4,
    "mei": 5,
    "juni": 6,
    "juli": 7,
    "augustus": 8,
    "september": 9,
    "oktober": 10,
    "november": 11,
    "december": 12
}

month_pattern = "(" + "|".join(month_map.keys()) + ")"

# Map typos to correct month names (based on all lines that give an error when parsing the date string)
rewrite_map = {
    'feburari': 'februari',
    'April': 'april',
    'junu': 'juni',
    'augsustus': 'augustus',
    'augusus': 'augustus',
    'augutus': 'augustus',
    'novemer': 'november',
    'spetember': 'september',
    'decenber': 'december',
    'ecember': 'december',
    'Augustus': 'augustus',
    'Februari': 'februari',
    'Februarij': 'februari',
    'Januari': 'januari',
    'Juli': 'juli',
    'Maart': 'maart',
    'Mei': 'mei',
    'September': 'september',
    'agustus': 'augustus',
    'arpil': 'april',
    'aug': 'augustus',
    'august': 'augustus',
    'augustys': 'augustus',
    'augusuts': 'augustus',
    'dcember': 'december',
    'decembe': 'december',
    'decmber': 'december',
    'decmeber': 'december',
    'febrari': 'februari',
    'ferbruari': 'februari',
    'ferbuari': 'februari',
    'feruari': 'februari',
    'gebruari': 'februari',
    'iuli': 'juli',
    'jan': 'januari',
    'januar': 'januari',
    'janurair': 'januari',
    'janurari': 'januari',
    'jui': 'juli',
    'juil': 'juli',
    'julie': 'juli',
    'julli': 'juli',
    'maarrt': 'maart',
    'meei': 'mei',
    'nomvember': 'november',
    'nov': 'november',
    'noveber': 'november',
    'nvovember': 'november',
    'okotber': 'oktober',
    'okotober': 'oktober',
    'oktber': 'oktober',
    'oktobrt': 'oktober',
    'paril': 'april',
    'seprember': 'september',
    'septelber': 'september',
    'septemer': 'september',
    'septemner': 'september',
    'setepmber': 'september',
    'setpember': 'september',
    'sptember': 'september',
    'uni': 'juni'
}




In [20]:
# First, extract all table rows
rows = [tr for tr in page_soup.find_all('tr')]
print('num rows:', len(rows))

# extra the header
header_row = rows[0]
headers = [td.text for td in header_row.find_all('td')]

# turn all HTML table rows into JSON dictionaries
data_rows = rows[1:]
rows_json = []

reference_row = {}

for di, data_row in enumerate(data_rows):
    cells = [td.text.strip() for td in data_row.find_all('td')]
    
    # pad rows with missing cells by adding empty strings
    if len(cells) < len(headers):
        extra = len(headers) - len(cells)
        cells = cells + [''] * extra

    # turn row to JSON
    row_json = {header: cells[hi] for hi, header in enumerate(headers)}
    row_json['row_idx'] = di
    author_id = row_json['ID']

    if 'zie:' in author_id:
        reference_row[di] = row_json
    # skip rows in the database of author IDs that are not in title metadata
    if author_id not in author_active.index:
        #print(row_json)
        continue

    # extract the active period and add it to the JSON dictionary
    active_start, active_end = get_active_years(author_id, author_active)
    row_json['active_start'], row_json['active_end'] = active_start, active_end

    # parse the Personalia field and update the row JSON with the DoB and DOD info
    personalia = parse_personalia(row_json)
    row_json.update(personalia)

    # add the row to the list of rows
    rows_json.append(row_json)


print(f"num selected rows: {len(rows_json)}")

num rows: 155590
num selected rows: 1335


In [21]:
print('number of authors with DoB, DoD and active period:', len(rows_json))

number of authors with DoB, DoD and active period: 1335


## Removing alternative names

E.g. Albert Helman used at least 15 different pen names. In the DBNL Auteurs dataset, such alternative/pen names are indicated by the next row containing a reference (`zie:`).

In [22]:
prev_row = None
filtered_rows = []
for ri, row in enumerate(rows_json):
    next_row_idx = row['row_idx'] + 1
    is_reference = False
    if next_row_idx in reference_row:
        #print(ri, row)
        #print(reference_row[next_row_idx])
        is_reference = True
        #break
    row['is_reference'] = is_reference
    if 'Helman' in row['Naam']:
        #print(row)
        pass
    if 'helm003' in row['ID']:
        print(row)
        #print()
        pass
    #print(row)
    #break
    

{'ID': 'helm003', 'Naam': 'Beckmesser', 'Volledige voornaam': '', 'Personalia': '', 'row_idx': 8125, 'active_start': 1926, 'active_end': 1999, 'dob_string': '', 'is_reference': True}
{'ID': 'helm003', 'Naam': 'Bentram-Matriotte, Hella', 'Volledige voornaam': '', 'Personalia': '', 'row_idx': 9469, 'active_start': 1926, 'active_end': 1999, 'dob_string': '', 'is_reference': True}
{'ID': 'helm003', 'Naam': 'Brandaris', 'Volledige voornaam': '', 'Personalia': '', 'row_idx': 18093, 'active_start': 1926, 'active_end': 1999, 'dob_string': '', 'is_reference': True}
{'ID': 'helm003', 'Naam': 'Helman, Albert', 'Volledige voornaam': '', 'Personalia': '(Paramaribo 7 november 1903 - Amsterdam 10 juli 1996)', 'row_idx': 56729, 'active_start': 1926, 'active_end': 1999, 'dob_string': 'Paramaribo 7 november 1903', 'dob_location': 'Paramaribo', 'dob_day': 7, 'dob_month': 11, 'dob_year': 1903, 'dob_century': 20.0, 'dod_string': 'Amsterdam 10 juli 1996', 'dod_location': 'Amsterdam', 'dod_day': 10, 'dod_mon

In [23]:
author_df = pd.DataFrame(rows_json)
author_df

,ID,Naam,Volledige voornaam,Personalia,row_idx,active_start,active_end,dob_string,is_reference,dob_location,dob_day,dob_month,dob_year,dob_century,dod_string,dod_location,dod_day,dod_month,dod_year,dod_century
0,meer010,"Aa, Martinus Wilhelmus van der",,,46,1869,1869,,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aalb003,"Aalberse, Han B.",,(Bruinisse 20 december 1917 - Bruinisse 12 jan...,77,1955,1955,Bruinisse 20 december 1917,False,Bruinisse,20.0,12.0,1917.0,20.0,Bruinisse 12 januari 1983,Bruinisse,12.0,1.0,1983.0,20.0
2,aals010,"Aalst, George van",,(1897 - 1925),108,1924,1924,1897,False,NaN,NaN,NaN,1897.0,19.0,1925,NaN,NaN,NaN,1925.0,20.0
3,kruy005,"Aalst-Gobius, Tine van",,,124,1900,1900,,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,aard002,"Aardweg, H.P. van den",[Hans Petrus],(Hoorn 19 juli 1899 - Amsterdam 4 juli 1971),171,1970,1970,Hoorn 19 juli 1899,False,Hoorn,19.0,7.0,1899.0,19.0,Amsterdam 4 juli 1971,Amsterdam,4.0,7.0,1971.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,koen006,"Zuyden, Annie van",,,153621,1913,1947,,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,zuyl002,"Zuylen, Belle van",,(Zuilen 20 oktober 1740 - Colombier 27 decembe...,153635,1985,1985,Zuilen 20 oktober 1740,False,Zuilen,20.0,10.0,1740.0,18.0,Colombier 27 december 1805,Colombier,27.0,12.0,1805.0,19.0
1332,zweg001,"Zwegers, Broeder Vergilius",,(1899 - 1976),153864,1950,1950,1899,False,NaN,NaN,NaN,1899.0,19.0,1976,NaN,NaN,NaN,1976.0,20.0
1333,zweg001,"Zwegers, Johan",,,153865,1950,1950,,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
author_df = author_df[author_df.is_reference == False]
author_df = author_df.drop(['row_idx', 'is_reference'], axis=1)
author_df = author_df.rename(columns={'ID': 'author_id'})
author_df.shape

(738, 18)

In [25]:
author_df[author_df.author_id.str.contains('kist00')]

,author_id,Naam,Volledige voornaam,Personalia,active_start,active_end,dob_string,dob_location,dob_day,dob_month,dob_year,dob_century,dod_string,dod_location,dod_day,dod_month,dod_year,dod_century
638,kist001,"Kist, Willem",,(Woerden 1 september 1758 - Arnhem 5 januari 1...,1800,1800,Woerden 1 september 1758,Woerden,1.0,9.0,1758.0,18.0,Arnhem 5 januari 1841,Arnhem,5.0,1.0,1841.0,19.0


Check whether we're missing any author IDs from the title metadata in the extended author info data:

In [26]:
author_set = set(author_df.author_id.unique())
[ai for ai in pub_df.author_id.unique() if ai not in author_set]


['wyth002', 'revi0028100', 'spie005no50']

We're missing info on three authors. I think we'll survive. 

In [27]:

author_df.to_csv('../metadata/dbnl-authors.tsv.gz', index=False, sep='\t', compression='gzip')

In [28]:
author_df.to_excel('../metadata/dbnl-authors.xlsx', index=False)

## Combining Publication and Author information

In [29]:
pub_df.shape, pub_df.drop_duplicates().shape

((2069, 27), (2069, 27))

In [30]:
author_df.author_id.value_counts()
author_df[author_df.author_id == 'helm003']

,author_id,Naam,Volledige voornaam,Personalia,active_start,active_end,dob_string,dob_location,dob_day,dob_month,dob_year,dob_century,dod_string,dod_location,dod_day,dod_month,dod_year,dod_century
522,helm003,"Helman, Albert",,(Paramaribo 7 november 1903 - Amsterdam 10 jul...,1926,1999,Paramaribo 7 november 1903,Paramaribo,7.0,11.0,1903.0,20.0,Amsterdam 10 juli 1996,Amsterdam,10.0,7.0,1996.0,20.0


In [31]:
pub_df[pub_df.author_id == 'helm003']

,ti_id,titel,jaar,druk,categorie,voornaam,voorvoegsel,achternaam,uitgever,plaats_van_uitgave,...,year_precision,decade_start,decade_end,is_anonymous,no_author_id,is_magazine,is_prose,reprint_id,orig_jaar,best_jaar
801,helm003zuid01,Zuid-Zuid-West,1926,1ste druk,werk,Albert,NaN,Helman,De Gemeenschap,[Utrecht],...,exact,1920,1920,False,False,False,True,NaN,NaN,1926
959,helm003hart01,Hart zonder land,1931,2de druk,herdruk,Albert,NaN,Helman,NaN,NaN,...,exact,1930,1930,False,False,False,True,helm003hart01,1929-1929,1929-1929
983,helm003euve01,Het euvel Gods,1932,1ste druk,werk,Albert,NaN,Helman,NaN,NaN,...,exact,1930,1930,False,False,False,True,NaN,NaN,1932
1003,helm003waar01,Waarom niet,1933,1ste druk,werk,Albert,NaN,Helman,Nijgh & Van Ditmar,Rotterdam,...,exact,1930,1930,False,False,False,True,NaN,NaN,1933
1035,helm003orka01,Orkaan bij nacht,1934,1ste druk,werk,Albert,NaN,Helman,NaN,NaN,...,exact,1930,1930,False,False,False,True,NaN,NaN,1934
1058,helm003doll01,De dolle dictator. Het ondoorgrondelijke leven...,1935,1ste druk,werk,Albert,NaN,Helman,NaN,NaN,...,exact,1930,1930,False,False,False,True,NaN,NaN,1935
1176,helm003ranc01,De rancho der X mysteries,1941,1ste druk,werk,Albert,NaN,Helman,Amsterdamsche Boek- en Courantmij,Amsterdam,...,exact,1940,1940,False,False,False,True,NaN,NaN,1941
1268,helm003klei01,Kleine kosmologie,1947,1ste druk,werk,Albert,NaN,Helman,Amsterdamsche Boek- en Courantmij,Amsterdam,...,exact,1940,1940,False,False,False,True,NaN,NaN,1947
1312,helm003afda01,Afdaling in de vulkaan,1949,1ste druk,werk,Albert,NaN,Helman,Amsterdamsche boek- en courantmaatschappij,Amsterdam,...,exact,1940,1940,False,False,False,True,NaN,NaN,1949
1351,helm003laai01,De laaiende stilte,1952,1ste druk,werk,Albert,NaN,Helman,Amsterdamsche boek- en courantmaatschappij,Amsterdam,...,exact,1950,1950,False,False,False,True,NaN,NaN,1952


In [32]:
[col for col in pub_df.columns if col in author_df.columns]

['author_id']

In [33]:
pub_author_df = pd.merge(pub_df, author_df, on='author_id')
pub_author_df.to_csv('../metadata/DBNL-publication_author_metadata.tsv', sep='\t', index=False)
pub_author_df.to_excel('../metadata/DBNL-publication_author_metadata.xlsx', index=False)



## Selecting Columns and Mapping Column Names

Use the columns "doc_id", "author_id" (also include author full name and surname), "birth_year", "death_year", "title", and "year_of_publication" (since it passed several filters and corrections, use "corrected_comp_year").


In [34]:
pub_author_df.columns

Index(['ti_id', 'titel', 'jaar', 'druk', 'categorie', 'voornaam',
       'voorvoegsel', 'achternaam', 'uitgever', 'plaats_van_uitgave',
       'cc_land', 'genre', 'geplaatst', 'link naar DBNL.org', 'author_id',
       'year_start', 'year_end', 'year_precision', 'decade_start',
       'decade_end', 'is_anonymous', 'no_author_id', 'is_magazine', 'is_prose',
       'reprint_id', 'orig_jaar', 'best_jaar', 'Naam', 'Volledige voornaam',
       'Personalia', 'active_start', 'active_end', 'dob_string',
       'dob_location', 'dob_day', 'dob_month', 'dob_year', 'dob_century',
       'dod_string', 'dod_location', 'dod_day', 'dod_month', 'dod_year',
       'dod_century'],
      dtype='object')

In [35]:
column_map = {
    'ti_id': 'doc_id',
    'titel': 'title',
    'author_id': 'author_id',
    'voornaam': 'first_name',
    'voorvoegsel': 'surname_prefix',
    'achternaam': 'surname',
    'Naam': 'full_name',
    'dob_year': 'birth_year',
    'dod_year': 'death_year',
    'year_start': 'year_of_publication'
}

pub_author_en = pub_author_df.rename(columns=column_map)[column_map.values()]
pub_author_en.to_csv('../metadata/DBNL-publication_author_metadata-english.tsv', index=False, sep='\t')

In [36]:
pub_author_en.doc_id.value_counts()

doc_id
kist001leve01    1
teir001mijn01    1
liuw002jell01    1
nijh002venu01    1
noor022kanm01    1
                ..
holt075scho01    1
gogh002hoog01    1
eman001mens01    1
coen011eila02    1
bosm029toch02    1
Name: count, Length: 2066, dtype: int64

In [37]:
pub_df.shape

(2069, 27)